# assignments

# Assignment 1: Groupby

Can you return a table containing the top 10 stores by total transactions in the data?

Make sure they’re sorted from highest to lowest.

Thanks!

In [1]:
import numpy as np
import pandas as pd
import matplotlib
print(matplotlib.__version__)  # to check if installed correctly


3.10.5


In [2]:
transactions = pd.read_csv(r"E:\Devu\USB Drive\coding\pandas-from-scratch\notebooks\Sec-3_DataFrameIntro\transactions.csv", parse_dates =["date"])
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [3]:
transactions.groupby("store_nbr")["transactions"].sum().head(10).sort_values( ascending =False).reset_index()

,store_nbr,transactions
0,3,5366350
1,8,4637971
2,9,3516162
3,2,3219901
4,6,3065896
5,7,2995993
6,1,2553963
7,4,2519007
8,5,2347877
9,10,1652493


# Assignment 2: Groupby Multiple Columns

Can you get me the total transactions by store and month? 

Sort the table from first month to last, then by highest transactions to lowest within each month. 


In [4]:
transactions["month"]=transactions["date"].dt.month


In [5]:
transactions.groupby(["store_nbr","month"],as_index=True)[["transactions"]].sum().sort_values(["month" , "transactions"], ascending=[True,False])

,,transactions
store_nbr,month,
44,1,628438
47,1,568824
45,1,538370
46,1,522763
3,1,463260
...,...,...
32,12,86167
21,12,84128
42,12,76741


# Assignment 3: Multi-Index DataFrames


Can you help me access rows and columns with multiple indices? I’ve been struggling with multi-index DataFrames.

Access:
* Grab Store 3, Month 1
* Then, select the column storing the mean of transactions

Fix:
* Drop the outer layer of the column Index
* Reset the row index so it is the default integer index

In [6]:
# Ross' grouped DataFrame code, run this first

grouped = (
    transactions.groupby(["store_nbr", "month"])
    .agg({"transactions": ["sum", "mean"]})
    .sort_values(by=["month", ("transactions", "sum")], ascending=[True, False])
)
grouped

transactions             
                         sum         mean
store_nbr month                          
44        1           628438  4246.202703
47        1           568824  3843.405405
45        1           538370  3637.635135
46        1           522763  3532.182432
3         1           463260  3151.428571
...                      ...          ...
32        12           86167   718.058333
21        12           84128  1402.133333
42        12           76741  1279.016667
29        12           76627  1277.116667
22        12           50650   844.166667

[641 rows x 2 columns]

In [7]:
grouped.loc[(3, 1)]


transactions  sum     463260.000000
              mean      3151.428571
Name: (3, 1), dtype: float64

In [8]:
grouped.loc[(3, 1), ("transactions", "mean")]


3151.4285714285716

In [9]:
grouped.columns = grouped.columns.droplevel(0)



In [10]:
grouped

,,sum,mean
store_nbr,month,,
44,1,628438,4246.202703
47,1,568824,3843.405405
45,1,538370,3637.635135
46,1,522763,3532.182432
3,1,463260,3151.428571
...,...,...,...
32,12,86167,718.058333
21,12,84128,1402.133333
42,12,76741,1279.016667


In [11]:
grouped.reset_index()

,store_nbr,month,sum,mean
0,44,1,628438,4246.202703
1,47,1,568824,3843.405405
2,45,1,538370,3637.635135
3,46,1,522763,3532.182432
4,3,1,463260,3151.428571
...,...,...,...,...
636,32,12,86167,718.058333
637,21,12,84128,1402.133333
638,42,12,76741,1279.016667
639,29,12,76627,1277.116667


# Assignment 4: The Agg Method

Calculate the mean of target met by store, and the sum of bonuses to be paid to each store.

Sort them by highest to lowest bonus payout.

Then, do the same for day of week and month.

In [12]:
# Recreate table from section 3

transactions = transactions.assign(
    target_pct=transactions["transactions"] / 2500,
    met_target=(transactions["transactions"] / 2500) >= 1,
    bonus_payable=((transactions["transactions"] / 2500) >= 1) * 100,
    month=transactions.date.dt.month,
    day_of_week=transactions.date.dt.dayofweek,
)

transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2


In [13]:
transactions.groupby(["store_nbr"]).agg(target_pct_mean=("target_pct","mean"),
                                                 bonus_sum=("bonus_payable","sum")).sort_values("bonus_sum" , ascending=False).head()

,target_pct_mean,bonus_sum
store_nbr,,
47,1.558929,167600
44,1.734787,167500
45,1.479097,167300
3,1.280752,167300
46,1.428769,165900


In [14]:
transactions.groupby(["month"]).agg(target_pct_mean=("target_pct","mean"),
                                                 bonus_sum=("bonus_payable","sum")).sort_values("bonus_sum" , ascending=False)

,target_pct_mean,bonus_sum
month,,
12,0.820027,154100
5,0.679678,131800
3,0.671496,130400
4,0.679770,129700
7,0.660090,126300
2,0.670462,121700
6,0.663416,121700
8,0.665931,120800
1,0.665637,119600


In [15]:

transactions.groupby(["day_of_week"]).agg(target_pct_mean=("target_pct","mean"),
                                                 bonus_sum=("bonus_payable","sum")).sort_values("bonus_sum" , ascending=False)


,target_pct_mean,bonus_sum
day_of_week,,
5,0.781070,266400
6,0.738904,241700
4,0.661596,213000
0,0.654223,191600
2,0.648599,191000
1,0.640523,175500
3,0.619880,169100


# Assignment 4: Transform

Calculate the mean of transactions by store number and day of week while keeping row numbers. 

Then compare the performance of each row to its day of week average. (difference between transactions and daily avg)

In [16]:
transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2


In [17]:
transactions.assign(row_numbers=(transactions.groupby("day_of_week")["store_nbr"].transform("mean")))

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week,row_numbers
0,2013-01-01,25,770,1,0.3080,False,0,1,26.941814
1,2013-01-02,1,2111,1,0.8444,False,0,2,26.934174
2,2013-01-02,2,2358,1,0.9432,False,0,2,26.934174
3,2013-01-02,3,3487,1,1.3948,True,100,2,26.934174
4,2013-01-02,4,1922,1,0.7688,False,0,2,26.934174
...,...,...,...,...,...,...,...,...,...
83483,2017-08-15,50,2804,8,1.1216,True,100,1,26.941814
83484,2017-08-15,51,1573,8,0.6292,False,0,1,26.941814
83485,2017-08-15,52,2255,8,0.9020,False,0,1,26.941814
83486,2017-08-15,53,932,8,0.3728,False,0,1,26.941814


# Assignment 5: Pivot

Pivot transactions with store number as index, columns day of week, with the sum of bonus payable as cells.

Filter to stores that had a non-zero bonus payable and create a heatmap.

Then unpivot (melt) the table so we have one row for each store and day of the week with the corresponding total owed. 


In [24]:
transactions.loc[transactions["bonus_payable"]!=0].pivot_table(index="store_nbr",
                         columns="day_of_week",
                         values="bonus_payable",
                         aggfunc = "sum",
                         margins=True).head().style.background_gradient(cmap="RdYlGn",axis=1)

                         
                         

day_of_week,0,1,2,3,4,5,6,All
store_nbr,,,,,,,,
1,200.000000,300.000000,300.000000,200.000000,100.000000,nan,nan,1100
2,300.000000,600.000000,500.000000,400.000000,400.000000,500.000000,200.000000,2900
3,24000.000000,23900.000000,23900.000000,23900.000000,23900.000000,24000.000000,23700.000000,167300
4,200.000000,300.000000,300.000000,200.000000,100.000000,200.000000,nan,1300
5,200.000000,300.000000,300.000000,100.000000,100.000000,100.000000,nan,1100


In [34]:
transactions.loc[transactions["bonus_payable"]!=0].pivot_table(index="store_nbr",
                         columns="day_of_week",
                         values="bonus_payable",
                         aggfunc = "sum",
                         margins=True).reset_index().melt(
                         id_vars="store_nbr",
                         value_name="bonus_payable"
                         )

,store_nbr,day_of_week,bonus_payable
0,1,0,200.0
1,2,0,300.0
2,3,0,24000.0
3,4,0,200.0
4,5,0,200.0
...,...,...,...
283,50,All,76000.0
284,51,All,3600.0
285,52,All,3800.0
286,53,All,300.0
